In [ ]:
import numpy as np
from pymatgen.io.vasp import Poscar
from pymatgen.core import Structure
from scipy.optimize import linear_sum_assignment

def objective_function(order, structure_A, structure_B, lattice):
    """
    Compute the sum of squared displacements between structure_A and reordered structure_B
    based on the given order.
    """
    reordered_structure_B = structure_B[order]
    total_distance = 0
    for i in range(len(structure_A)):
        frac_distance = structure_A[i] - reordered_structure_B[i]
        frac_distance = frac_distance - np.round(frac_distance)  # Apply PBC
        cart_distance = lattice.get_cartesian_coords(frac_distance)
        total_distance += np.dot(cart_distance, cart_distance)
    return total_distance

folder_path = "./"
poscar_file_A = folder_path + '/CONTCAR_B_VGa_tuned.vasp'
poscar_file_B = folder_path + '/CONTCAR_G_VGa_tuned.vasp'

structure_A = Poscar.from_file(poscar_file_A).structure.frac_coords
structure_B = Poscar.from_file(poscar_file_B).structure.frac_coords
lattice_A = Poscar.from_file(poscar_file_A).structure.lattice

num_atoms = len(structure_A)
cost_mat = np.zeros((num_atoms, num_atoms))

for i in range(num_atoms):
    for j in range(num_atoms):
        frac_distance = structure_A[i] - structure_B[j]
        frac_distance = frac_distance - np.round(frac_distance)  # Apply PBC
        cart_distance = lattice_A.get_cartesian_coords(frac_distance)
        cost_mat[i, j] = np.dot(cart_distance, cart_distance)

row_ind, col_ind = linear_sum_assignment(cost_mat)

minimum_cost = objective_function(col_ind, structure_A, structure_B, lattice_A)
print(f"Minimum total squared displacement: {minimum_cost}")

reordered_structure_B = structure_B[col_ind]

lattice_B = Poscar.from_file(poscar_file_B).structure.lattice
reordered_structure_B_struct = Structure(
    lattice=lattice_B,
    species=["Ga"] * len(reordered_structure_B),
    coords=reordered_structure_B,
    coords_are_cartesian=False
)

output_file = folder_path + '/Results_VGa/Optimal_CONTCAR.vasp'
Poscar(reordered_structure_B_struct).write_file(output_file)
print(f"Reordered structure written to: {output_file}")
